# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime


In [5]:
#Define folder path

folderpath = r'C:\Users\ANITA BOADU\CitiBike_Dashboard_Project\Data\Original Data'

# Step 2: Get ONLY the CSV files 
filepaths = [os.path.join(folderpath, f) for f in os.listdir(folderpath) if f.endswith('.csv')]

if filepaths:
    df = pd.concat((pd.read_csv(f, low_memory=False) for f in filepaths), ignore_index=True)
    print(f"Successfully merged {len(filepaths)} files.")
    display(df.head())
else:
    print("No CSV files found in the directory. Check your file extensions!")

Successfully merged 18 files.


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7830EBE775F2EE25,classic_bike,2021-06-21 07:38:42,2021-06-21 10:49:25,Dixon Mills,JC076,NaN,NaN,40.721630,-74.049968,NaN,NaN,member
1,629A3BFD50BF8392,docked_bike,2021-06-12 10:25:01,2021-06-12 10:35:11,Dixon Mills,JC076,Journal Square,JC103,40.721630,-74.049967,40.733670,-74.062500,member
2,CF02EC434989A99D,docked_bike,2021-06-07 14:52:05,2021-06-07 14:58:25,11 St & Washington St,HB502,Hoboken Terminal - Hudson St & Hudson Pl,HB101,40.749984,-74.027150,40.735938,-74.030305,member
3,E1BB8FED29AA5BA1,docked_bike,2021-06-07 08:29:19,2021-06-07 08:34:17,8 St & Washington St,HB603,Hoboken Terminal - Hudson St & Hudson Pl,HB101,40.745983,-74.028199,40.735938,-74.030305,casual
4,787C59EE76D96BCA,docked_bike,2021-06-18 14:16:27,2021-06-18 14:26:26,11 St & Washington St,HB502,Columbus Park - Clinton St & 9 St,HB501,40.749984,-74.027150,40.748161,-74.032453,member


In [6]:
import requests
import json
from datetime import datetime

# NOAA Token
Token = "ZhnpynsbAVtURYtUshPNuYGAFSiurmGe"

# URL for LaGuardia Station
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31"

r = requests.get(url, headers={'token': Token})
d = json.loads(r.text)

# Extract temperature data
avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

dates_temp = [item['date'] for item in avg_temps]
temps = [item['value'] for item in avg_temps]

df_temp = pd.DataFrame()
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]  # convert tenths of °C to °C

# Export weather data
df_temp.to_csv("nyc_weather_2022.csv", index=False)


In [7]:
# Convert trip datetime to date only
df['started_at'] = pd.to_datetime(df['started_at'])
df['date'] = df['started_at'].dt.date

df_temp['date'] = df_temp['date'].dt.date

# Merge on 'date'
df_merged = df.merge(df_temp, how='left', on='date')

# Export merged dataset
df_merged.to_csv("citibike_2022_with_weather.csv", index=False)
